In [3]:
import cv2 as cv
import numpy as np
import os
from sklearn import metrics

In [11]:
def detect(path_img,threshold):

    #Input
    img = cv.imread(path_img)
    size = len(img)

    #Grayscale
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    #Thresholding
    ret,img = cv.threshold(img, threshold, 255, cv.THRESH_BINARY)
    
    #Mask
    mask = np.zeros((size,size,3), np.uint8)
    c = int(size/2)
    cv.ellipse(mask,(c,c-100),(100,100),0,0,360,(255,255,255),5)
    mask = cv.cvtColor(mask, cv.COLOR_BGR2GRAY)

    #Pixel Matching
    result = cv.bitwise_and(img,img,mask=mask)
    matched = sum(sum(result))*100/sum(sum(mask))

    #Edge prewit
    
    #Result
    # window = np.concatenate((img,mask,result), axis=1)
    # cv.imshow("window",window)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    return matched

In [15]:
ds = []
pathu = r"C:\Users\y\code\ai\u"
pathnu = r"C:\Users\y\code\ai\nu"
for img_path in os.listdir(pathu):
    ds.append(pathu+'\\'+img_path)
for img_path in os.listdir(pathnu):
    ds.append(pathnu+'\\'+img_path)

pred = [ds[i].split('\\')[-2]=='u' for i in range(len(ds))]
res = [detect(ds[i],100,54)>54 for i in range(len(ds))]

print(metrics.classification_report(pred,res))
print(metrics.confusion_matrix(pred,res))

              precision    recall  f1-score   support

       False       0.77      0.76      0.76       230
        True       0.56      0.57      0.57       124

    accuracy                           0.69       354
   macro avg       0.67      0.67      0.67       354
weighted avg       0.70      0.69      0.70       354

